20241222 创建repo的过程:

In [ ]:
!git clone --recurse-submodules https://github.com/tdrussell/diffusion-pipe


!conda create -n diffusion-pipe python=3.12
!conda activate diffusion-pipe

!pip install xformers "torch<2.4" torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 -U 
!pip install -r requirements.txt

move model weights to local:

In [1]:
!ls  /rmd/yada/model_weights/

aesthetic-shadow-v2	complexity_ic9600_ck.pth  hydit_v1.2_kohya
clip-vit-large-patch14	convnextv2-base-22k-384   style_emb


In [7]:
!ls

aesthetic-shadow-v2	  hunyuan_video_t2v_720p_bf16.safetensors
clip-vit-large-patch14	  hydit_v1.2_kohya
complexity_ic9600_ck.pth  llava-llama-3-8b-text-encoder-tokenizer
convnextv2-base-22k-384   style_emb


In [21]:
!mkdir -p /lv0/yada/tmp_dev
%cd /rmd/yada/model_weights/
!ls

/rmd/yada/model_weights
aesthetic-shadow-v2	  hunyuan_video_t2v_720p_bf16.safetensors
clip-vit-large-patch14	  hydit_v1.2_kohya
complexity_ic9600_ck.pth  llava-llama-3-8b-text-encoder-tokenizer
convnextv2-base-22k-384   style_emb


In [22]:
!du -sh llava-llama-3-8b-text-encoder-tokenizer

30G	llava-llama-3-8b-text-encoder-tokenizer


In [ ]:
# 3 minutes
!cp -r clip-vit-large-patch14 llava-llama-3-8b-text-encoder-tokenizer  /lv0/yada/tmp_dev/

In [25]:
# 3.5 minutes
!cp hunyuanVideoSafetensors_vae.safetensors hunyuan_video_t2v_720p_bf16.safetensors /lv0/yada/tmp_dev/

running trainer (in diffusion-pipe repo):

In [ ]:
deepspeed --num_gpus=1 train.py --deepspeed --config personal_files/test_hunyuan_video.toml --regenerate_cache

deepspeed --num_gpus=1 train.py --deepspeed --config personal_files/test_hunyuan_video.toml

deepspeed --num_gpus=1 train.py --deepspeed --config personal_files/test_hunyuan_video_pixel.toml --regenerate_cache

deepspeed --num_gpus=8 train.py --deepspeed --config personal_files/test_hunyuan_video_illust.toml


deepspeed --num_gpus=6 train.py --deepspeed --config personal_files/test_hunyuan_video_illust.toml


## comfyui:

In [ ]:
conda create -n comfyui python=3.12

# torch 2.5.1
pip install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu124

In [ ]:
# training_flow / setup_comfy
import os

def find_requirements_dirs(base_dir):
    # Find all direct subdirectories with a requirements.txt
    dirs_with_requirements = [
        os.path.join(base_dir, d)
        for d in os.listdir(base_dir)
        if os.path.isdir(os.path.join(base_dir, d)) and os.path.exists(os.path.join(base_dir, d, 'requirements.txt'))
    ]
    return dirs_with_requirements

def generate_install_command(requirements_dirs):
    # Generate the install command for each directory
    command = ' && '.join(
        [f'pip install -r {os.path.join(d, "requirements.txt")}' for d in requirements_dirs]
    )
    return command

base_directory = '/rmt/yada/apps/comfyui/custom_nodes'
requirements_dirs = find_requirements_dirs(base_directory)

if requirements_dirs:
    install_command = generate_install_command(requirements_dirs)
    print(install_command)
else:
    print("No subdirectories with requirements.txt found.")